In [1]:
# inspired by https://github.com/pommi/python-itho-wpu/blob/master/convert-itho-db.py

PARFILE = ".\parameters_HeatPump.mdb"
PASSWORD = 'itho_parameters'

import pyodbc, re
from collections import defaultdict
import slugify

In [2]:
# import "hand translated" StatusLabels

import handmadelabels
handmadelabels.StatusLabels[:5]

[('Outside temp (°C)', 'outside-temp_c'),
 ('Boiler temp down (°C)', 'boilertemp-down_c'),
 ('Boiler temp up (°C)', 'boilertemp-up_c'),
 ('Evaporator temp (°C)', 'evaporator-temp_c'),
 ('Suction gas temp (°C)', 'suction-gas-temp_c')]

In [3]:
# zoek de driver
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

['Microsoft Access Driver (*.mdb, *.accdb)']

## Lees MDB

In [4]:
MDB = PARFILE
DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'


# connect to db
conn = pyodbc.connect(f'DRIVER={DRV};DBQ={PARFILE};PWD={PASSWORD}')
conn.setencoding("UTF-8")
conn.setdecoding(pyodbc.SQL_CHAR, encoding="UTF-8")
cur = conn.cursor()


In [5]:
tables = []
for table_info in cur.tables(tableType="TABLE"):
        if re.match(
            "^(VersieBeheer|Data[Ll]abel|Parameterlijst|Handbed|Counters)", table_info.table_name
        ):
            tables.append(table_info.table_name)
tables[:5]

['Counters_V10',
 'Counters_V2',
 'Counters_V37',
 'DataLabel_V10',
 'DataLabel_V11']

In [6]:
def get_version(s):
    x = re.findall('\d+', s)
    assert len(x) == 1
    return int(x[0])

assert get_version("Parameterlijst_V10")==10

In [7]:
WPUSettingsLabels = defaultdict(list)
WPUStatusLabels = defaultdict(list)
Versies = []

StatusLabelNames = []  # "Naam" column in DataLabel_Vxx tables StatusLabelName['T_out'] = 0
StatusLabels = {}  # StatusLabels['T_out'] = ('Outside Temp (C)', 'outside-temp-c') 


for table in sorted(tables):
    #print(table)
    
    if re.match("^Parameterlijst", table):
        fw_ver = get_version(table)
        cur.execute(f"select Index, Naam, Tekst_NL, Tekst_GB, Eenheid_NL, Eenheid_GB from {table}") 
        rows = cur.fetchall()
        for r in sorted(rows):
            WPUSettingsLabels[fw_ver].append((r.Index, r.Naam, r.Tekst_NL, r.Eenheid_NL, r.Tekst_GB, r.Eenheid_GB))
    if re.match("^Data[Ll]abel", table):
        fw_ver = get_version(table)
        cur.execute(f"select Index, Naam, Tekst_NL, Tekst_GB, Eenheid_NL, Eenheid_GB from {table}")
        rows = cur.fetchall()
        for r in sorted(rows):
            WPUStatusLabels[fw_ver].append((r.Index, r.Naam, r.Tekst_NL, r.Eenheid_NL, r.Tekst_GB, r.Eenheid_GB))          
    if re.match("^VersieBeheer", table):
        cur.execute(f"select VersieNummer, DataLabel, ParameterLijst, Handbed, Counters from {table}")
        rows = cur.fetchall()
        for r in sorted(rows):
            Versies.append((r.VersieNummer, r.DataLabel, r.ParameterLijst, r.Handbed, r.Counters))        
#sorted(WPUSettingsLabels.keys()), sorted(WPUStatusLabels.keys())    

In [8]:
# get all versions of tables for each firmware

fw_versions = {}

DataLabels = {}
SettingLabels = {}
for versie in Versies:
    fw, datalabel, setting, handbed, counter = versie
    DataLabels[datalabel] = datalabel
    SettingLabels[setting] = setting
fw_versions['datalabels']= sorted([int(x) for x in DataLabels.keys()])
fw_versions['settings'] = sorted([int(x) for x in SettingLabels.keys()])

print(fw_versions)

{'datalabels': [2, 4, 5, 6, 7, 8, 10, 11, 18, 20, 21, 22, 23, 24, 25, 27, 31, 33, 34, 37, 41], 'settings': [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 31, 33, 34, 37, 41]}


## Data Labels

In [9]:
WPUStatusLabels[2][:5]

[(0, 'T_out', 'Buitentemp', '°C', None, None),
 (1, 'T_BoilDwn', 'Boiler laag', '°C', None, None),
 (2, 'T_BoilTop', 'Boiler hoog', '°C', None, None),
 (3, 'T_Evap', 'Verdamper temp', '°C', None, None),
 (4, 'T_Suct', 'Zuiggas temp', '°C', None, None)]

In [10]:
def itho_WPU_status_line(fw_ver, l):
    indices_str = ', '.join(str(x) for x in l)
    return f'const uint8_t itho_WPUstatus{fw_ver}[]{{{indices_str}, 255}};'

print(itho_WPU_status_line(42, [3, 99, 42]))

const uint8_t itho_WPUstatus42[]{3, 99, 42, 255};


In [11]:
# Generate the index tables 'itho_WPUstatus' from the parameter tables
def print_itho_WPU_status_lines(fw_versions, WPUStatusLabels):
    StatusLabelNames = []
    for fw_ver in fw_versions['datalabels']:
        fw_ver = int(fw_ver)
        #print(fw_ver)
        label_list = []
        for label in WPUStatusLabels[fw_ver]:
            idx, name, _,_,_,_ = label
            #print(idx, name)
            if name not in StatusLabelNames:
                StatusLabelNames.append(name)
            list_idx = StatusLabelNames.index(name)
            label_list.append(list_idx)
        print(itho_WPU_status_line(fw_ver, label_list))
    return StatusLabelNames

print("// diff this against wpu.h. Only 8_9 11_17 and 18_19 are different. All keys should be the same")
StatusLabelNames = print_itho_WPU_status_lines(fw_versions, WPUStatusLabels)


// diff this against wpu.h. Only 8_9 11_17 and 18_19 are different. All keys should be the same
const uint8_t itho_WPUstatus2[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 255};
const uint8_t itho_WPUstatus4[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 255};
const uint8_t itho_WPUstatus5[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 68, 255};
const uint8_t itho_WPUstatus6[]{0, 1, 2, 3, 4, 5, 6, 7, 

In [22]:
def create_lookup_table_statuslabels(fw_versions, WPUStatusLabels):
    Labels = {}

    # create lookup tabel for Statuslabels, based on most recent firmware
    for fw in fw_versions:
        for label in WPUStatusLabels[fw]:
            idx, name, nl_tekst, nl_unit, gb_tekst, gb_unit = label
            #print(name, gb_tekst, gb_unit)
            if gb_tekst is None:
                gb_tekst = nl_tekst
                gb_unit = nl_unit
            if gb_unit is None:
                unit = ""
            else:
                unit = " ("+gb_unit+")"
            desc = gb_tekst+unit
            # create a slug. Hand generated slugs use C instead of degc for temperature units
            slug = str(slugify.slugify(desc)).replace('degc', 'c')
            Labels[name] = (desc,slug)
    return Labels

Labels = create_lookup_table_statuslabels(fw_versions['datalabels'], WPUStatusLabels)
Labels['T_out']

('Outside temp (°C)', 'outside-temp-c')

In [23]:
print("const struct ithoLabels ithoWPUStatusLabels[] { \t//index".expandtabs(120))
for idx,label in enumerate(StatusLabelNames):
    desc, slug = Labels[label]
    print(f'    {{"{desc}", "{slug}"}},\t//{idx}'.expandtabs(120))

const struct ithoLabels ithoWPUStatusLabels[] {                                                                         //index
    {"Outside temp (°C)", "outside-temp-c"},                                                                            //0
    {"boilertemp under (°C)", "boilertemp-under-c"},                                                                    //1
    {"boilertemp above (°C)", "boilertemp-above-c"},                                                                    //2
    {"evaporator temp (°C)", "evaporator-temp-c"},                                                                      //3
    {"suction gas temp (°C)", "suction-gas-temp-c"},                                                                    //4
    {"compressed gas (°C)", "compressed-gas-c"},                                                                        //5
    {"liquid temp (°C)", "liquid-temp-c"},                                                                              //6
    

In [24]:
print("const struct ithoLabels ithoWPUStatusLabels[] { \t//index".expandtabs(120))
for idx,label in enumerate(StatusLabelNames):
    try:
        # replace with the "old" handmade translation
        desc, slug = handmadelabels.StatusLabels[idx]
    except IndexError:
        # new key/value!
        desc, slug = Labels[label]
        print('// new label: ', desc, slug)
    print(f'    {{"{desc}", "{slug}"}},\t//{idx}'.expandtabs(120))

const struct ithoLabels ithoWPUStatusLabels[] {                                                                         //index
    {"Outside temp (°C)", "outside-temp_c"},                                                                            //0
    {"Boiler temp down (°C)", "boilertemp-down_c"},                                                                     //1
    {"Boiler temp up (°C)", "boilertemp-up_c"},                                                                         //2
    {"Evaporator temp (°C)", "evaporator-temp_c"},                                                                      //3
    {"Suction gas temp (°C)", "suction-gas-temp_c"},                                                                    //4
    {"Compressed gas temp (°C)", "compressed-gas-temp_c"},                                                              //5
    {"Liquid temp (°C)", "liquid-temp_c"},                                                                              //6
    

## SettingLabels

In [15]:
print(fw_versions['settings'])

[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 31, 33, 34, 37, 41]


In [16]:
def itho_WPU_setting_line(fw_ver, l):
    indices = ', '.join(str(x) for x in l)
    return f'const uint16_t itho_WPUsetting{fw_ver}[]{{{indices}, 999}};'

print(itho_WPU_setting_line(42, [3, 99, 42]))

const uint16_t itho_WPUsetting42[]{3, 99, 42, 999};


In [17]:
def print_itho_WPUsetting(fw_versions, WPUSettingsLabels):
    SettingLabelNames = []
    for fw_ver in fw_versions['settings']:
        fw_ver = int(fw_ver)
        #print(fw_ver)
        label_list = []
        for label in WPUSettingsLabels[fw_ver]:
            idx, name, _, _, _, _ = label
            #print(idx, name)
            if name not in SettingLabelNames:
                SettingLabelNames.append(name)
            list_idx = SettingLabelNames.index(name)
            label_list.append(list_idx)
        print(itho_WPU_setting_line(fw_ver, label_list))
    return SettingLabelNames

print("// diff this against wpu.h.")
SettingLabelNames = print_itho_WPUsetting(fw_versions, WPUSettingsLabels)

// diff this against wpu.h.
const uint16_t itho_WPUsetting2[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 999};
const uint16_t itho_WPUsetting4[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 108, 109, 110, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 111, 56, 57, 112, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 113, 114, 115, 116, 117, 118, 119, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 

In [18]:
def create_lookup_table_settings(fw_versions, WPUSettingsLabels):
    Labels = {}

    # create lookup tabel for Statuslabels, based on most recent firmware
    for fw in fw_versions:
        for label in WPUSettingsLabels[fw]:
            idx, name, nl_tekst, nl_unit, gb_tekst, gb_unit = label
            #print(name, gb_tekst, gb_unit)
            if gb_tekst is None:
                gb_tekst = nl_tekst
                gb_unit = nl_unit
            if gb_unit is None:
                unit = ""
            else:
                unit = " ("+gb_unit+")"
            desc = gb_tekst+unit
            Labels[name] = desc
    return Labels

Labels = create_lookup_table_settings(fw_versions['settings'], WPUSettingsLabels)
Labels['Spare']

'Spare'

In [19]:
print("const char *ithoWPUSettingsLabels[] = { \t//index".expandtabs(70))
for idx,label in enumerate(SettingLabelNames):
    desc = Labels[label]
    print(f"    \"{desc}\",\t//{idx}".expandtabs(70))

const char *ithoWPUSettingsLabels[] = {                               //index
    "niet gebruikt",                                                  //0
    "Hardware configuration",                                         //1
    "Year of commissioning",                                          //2
    "Date of commissioning",                                          //3
    "max manaul opeation time (min)",                                 //4
    "frost temp (°C)",                                                //5
    "offset for forst temp (K)",                                      //6
    "differential vorst temp electrical element (K)",                 //7
    "Error reset time (min)",                                         //8
    "Log interval (sec)",                                             //9
    "switch-on delay (sec)",                                          //10
    "Speed expansion valve (ticks)",                                  //11
    "Max open expansion valve (p